In [4]:
from bert_serving.client import BertClient 
bc = BertClient(check_length=False)

In [66]:
import pandas as pd
import numpy as np
from setup import get_train, get_test, get_valid
train6 = get_train(6)
test6 = get_test(6)

In [67]:
train3 = get_train(3)
test3 = get_test(3)
train2 = get_train(2)
test2 = get_test(2)

In [68]:
X_str_train = train6['statement']
y_train6 = train6['label']
X_str_test = test6['statement']
y_test6 = test6['label']

y_train3 = get_train(3)['label']
y_test3 = get_test(3)['label']
y_train2 = get_train(2)['label']
y_test2 = get_test(2)['label']

In [129]:
#bert-as-a-service recommends not pre-batching the inputs, since the program does this automatically
#however, despite reading "sent back size 10240" after the job completes, the jupyter cell still hangs
#batching prevents this issue, and lets the BERT encoding run to completion successfully
ys = []
n = len(X_str_train)
for i in range(n):
    y, _ = bc.encode(
        X_str_train.tolist()[i:(i+1)], show_tokens=True)
    ys.append(y[0])

In [130]:
#manually pad and concatenate the sentence by sentence encodings
m = max(y.shape[0] for y in ys)
X_bert_train = np.zeros((n, m, 768))
for i in range(len(ys)):
    y = ys[i]
    X_bert_train[i] = np.pad(y, ((0, m-y.shape[0]), (0,0)), 'constant')

In [131]:
#np.save('data/bert/X_bert_train', X_bert_train)

In [ ]:
#X_Bert_train = np.load('data/bert/X_bert_train.npy')
#remove pre-mean bert_train to save space

In [132]:
#average the sentence level encodings into a single 768-dimension vector
def bert_reduce_mean(X):
    return X.mean(axis=1) 

In [450]:
#X_bert_train_mean = bert_reduce_mean(X_bert_train)
#np.save('data/bert/X_bert_train_mean', X_bert_train_mean)
X_bert_train_mean = np.load('data/bert/X_bert_train_mean.npy')

In [1]:
import os
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNClassifier
from sklearn.metrics import classification_report

In [428]:
mod1 = TorchShallowNeuralClassifier(
    max_iter=110, hidden_dim=500)
#changing learning rate (eta) and 
#l2 regularization (l2_strength) suboptimum 
#for this model space at parameters other than default

In [429]:
y_train = y_train6
%time _ = mod1.fit(X_bert_train_mean, tuple(y_train.tolist()))

Finished epoch 110 of 110; error is 14.775890350341797

CPU times: user 4min 23s, sys: 1min 14s, total: 5min 37s
Wall time: 32.4 s


In [143]:
ys2 = []
n2 = len(X_str_test)
for i in range(n2):
    y, _ = bc.encode(
        X_str_test.tolist()[i:(i+1)], show_tokens=True)
    ys2.append(y[0])
#perform encoding for the test set

In [144]:
m2 = max(y.shape[0] for y in ys)
X_bert_test = np.zeros((n2, m2, 768))
for i in range(len(ys2)):
    y = ys2[i]
    X_bert_test[i] = np.pad(y, ((0, m2-y.shape[0]), (0,0)), 'constant')

In [145]:
#np.save('data/bert/X_bert_test', X_bert_test)
#X_bert_test = np.load('data/bert/X_bert_test.npy')

In [15]:
#X_bert_test_mean = bert_reduce_mean(X_bert_test)
#np.save('data/bert/X_bert_test_mean', X_bert_test_mean)
X_bert_test_mean = np.load('data/bert/X_bert_test_mean.npy')
X_bert_train_mean = np.load('data/bert/X_bert_train_mean.npy')

In [16]:
X_bert_train_mean.shape

(10240, 768)

In [ ]:
bert_train_preds = mod.predict(X_bert_train_mean)
print(classification_report(y_train, bert_train_preds, digits=3))
bert_test_preds = mod.predict(X_bert_test_mean)
print(classification_report(y_test, bert_test_preds, digits=3))

In [13]:
extended_train = pd.read_csv('data/liar/liwc_train.csv')
extended_test = pd.read_csv('data/liar/liwc_test.csv')
#pre-computed LIWC-2015 corpus feature-level counts on each statement. 
#Extends the columns by ~100 features (in addition to meta-data)

In [58]:
def extend_train(subset):
    train_e = np.ndarray((extended_train.shape[0], X_bert_train_mean.shape[1]+len(subset)))
    for j in range(extended_train.shape[0]):
        train_e[j] = np.append(X_bert_train_mean[j], extended_train[subset].values[j])
    return train_e

def extend_test(subset):
    test_e = np.ndarray((extended_test.shape[0], X_bert_test_mean.shape[1]+len(subset)))
    for j in range(extended_test.shape[0]):
        test_e[j] = np.append(X_bert_test_mean[j], extended_test[subset].values[j])
    return test_e

In [59]:
all_liwc_labels = extended_train.columns[15:]

In [463]:
#extend the BERT feature vectors with the LIWC features
#X_bert_train_liwc = extend_train(all_liwc_labels)
#np.save('data/bert/X_bert_train_liwc', X_bert_train_liwc)
#X_bert_test_liwc = extend_test(all_liwc_labels)
#np.save('data/bert/X_bert_test_liwc', X_bert_test_liwc)
X_bert_test_liwc = np.load('data/bert/X_bert_test_liwc.npy')
X_bert_train_liwc = np.load('data/bert/X_bert_train_liwc.npy')

In [465]:
#extend only with the three feature functions of interest from Newman et al.
subset = ['ppron', 'negemo', 'cogproc']
#X_bert_train_liwc_rest = extend_train(subset)
#np.save('data/bert/X_bert_train_liwc_rest', X_bert_train_liwc_rest)
#X_bert_test_liwc_rest = extend_test(subset)
#np.save('data/bert/X_bert_test_liwc_rest', X_bert_test_liwc_rest)
X_bert_train_liwc_rest = np.load('data/bert/X_bert_train_liwc_rest.npy')
X_bert_test_liwc_rest = extend_test('data/bert/X_bert_test_liwc.npy')

In [60]:
#extend with a random subset of 10 feature functions
chosen = 10
subset = np.random.choice(extended_train.columns[15:], chosen)

X_bert_train_liwc_rand = extend_train(subset)
X_bert_test_liwc_rand = extend_test(subset)

In [48]:
mod2 = TorchShallowNeuralClassifier(
    max_iter=100, hidden_dim=500)

In [49]:
#set the training set to be the extension with featureset of choice, and choose y as desired (number of labels)
train_wbl = X_bert_train_liwc_rand
test_wbl = X_bert_test_liwc_rand
y_train = y_train6
%time _ = mod1.fit(train_wbl, tuple(y_train.tolist()))

Finished epoch 100 of 100; error is 7.013440847396851

CPU times: user 3min 35s, sys: 35.7 s, total: 4min 11s
Wall time: 23.2 s


In [63]:
bert_train_preds = mod1.predict(train_wbl)
bert_test_preds = mod1.predict(test_wbl)

In [64]:
print(classification_report(y_train, bert_train_preds, digits=3))
print(classification_report(y_test, bert_test_preds, digits=3))

              precision    recall  f1-score   support

           0      0.903     0.746     0.817       839
           1      0.838     0.751     0.792      1995
           2      0.842     0.747     0.792      1654
           3      0.852     0.750     0.797      2114
           4      0.846     0.696     0.764      1962
           5      0.553     0.930     0.694      1676

   micro avg      0.768     0.768     0.768     10240
   macro avg      0.806     0.770     0.776     10240
weighted avg      0.802     0.768     0.774     10240

              precision    recall  f1-score   support

           0      0.059     0.011     0.018        92
           1      0.216     0.301     0.252       249
           2      0.135     0.160     0.147       212
           3      0.216     0.468     0.295       265
           4      0.158     0.012     0.023       241
           5      0.190     0.053     0.083       208

   micro avg      0.196     0.196     0.196      1267
   macro avg      0.162